# Import necessary packages and change display settings

In [ ]:
import qgrid
import plotly.graph_objects as go
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format # display numbers as with thousands separator
pd.set_option('display.max_colwidth', None) # display full text in columns

from IPython.display import display, HTML

# Custom CSS to align DataFrame values to the right
css_rules = """
<style>
    table.dataframe th, table.dataframe td {
        text-align: right !important;
    }
</style>
"""

# Apply the custom CSS
display(HTML(css_rules))

## Import pre-cleaned data and find most viable "EARN" variables

In [ ]:
df = pd.read_csv('Data/clean_field_of_study.csv', dtype={'CIPFIELD': str, 'CIPCODE': str})

# Look at which "EARN" variables have the most datapoints
earn_col_counts = {}
for col in df.columns:
    if "EARN" in col:
        earn_col_counts[col] = len(df[col].unique()) -1 # subtract 1 to account for NaN

# sort dictionary by value to see which variables have the most data
earn_col_counts = {k: v for k, v in sorted(earn_col_counts.items(), key=lambda item: item[1], reverse=True)}
list(earn_col_counts.items())[:5] # Display first 5 items

## Filter data down to the key variables discovered above
* EARN_MDN_1YR: 'Median earnings of graduates working and not enrolled 1 year after completing'   
* EARN_MDN_4YR: 'Median earnings of graduates working and not enrolled 4 years after completing'   
* EARN_COUNT_WNE_1YR: 'Number of graduates working and not enrolled 1 year after completing'   
* EARN_COUNT_WNE_4YR: 'Number of graduates working and not enrolled 4 years after completing'

In [ ]:
# We decide to us EARN_MDN_1YR and EARN_MDN_4YR
df = df[df.columns.to_list()[:7] + ['EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_COUNT_WNE_1YR', 'EARN_COUNT_WNE_4YR']] 

# Get counts of nans
# Number of rows where all four EARN columns have NaN values
all_earn_nan = df[df[['EARN_MDN_1YR', 'EARN_MDN_4YR', 'EARN_COUNT_WNE_1YR', 'EARN_COUNT_WNE_4YR']].isna().all(axis=1)].shape[0]

# Number of rows where each individual EARN column has NaN values
earn_mdn_1yr_nan = df['EARN_MDN_1YR'].isna().sum()
earn_mdn_4yr_nan = df['EARN_MDN_4YR'].isna().sum()
earn_count_wne_1yr_nan = df['EARN_COUNT_WNE_1YR'].isna().sum()
earn_count_wne_4yr_nan = df['EARN_COUNT_WNE_4YR'].isna().sum()

print(f'All four EARN columns nan count: {all_earn_nan}\n1YR median earnings nan count: {earn_mdn_1yr_nan}\n4YR median earnings nan count: {earn_mdn_4yr_nan}')

# Drop rows with nan's (we lose 310 total rows -- 262 of which all four columns are nan)
df.dropna(inplace=True)
df.to_csv('Data/ouchie.csv', index=False)


## Explore earnings by University

In [ ]:
# Group earnings by universities (summary stats of all programs with public record)
univ_1yr_stats = df.groupby('INSTNM').agg({'EARN_COUNT_WNE_1YR': 'sum','EARN_MDN_1YR': ['mean', 'min', 'max']}).reset_index()
univ_1yr_stats = univ_1yr_stats.sort_values(by=('EARN_MDN_1YR', 'mean'), ascending=False).reset_index(drop=True)
univ_1yr_stats

In [ ]:
# Group earnings by universities (summary stats of all programs with public record)
univ_4yr_stats = df.groupby('INSTNM').agg({'EARN_COUNT_WNE_4YR': 'sum','EARN_MDN_4YR': ['mean', 'min', 'max']}).reset_index()
univ_4yr_stats = univ_4yr_stats.sort_values(by=('EARN_MDN_4YR', 'mean'), ascending=False).reset_index(drop=True)
univ_4yr_stats

## Visualize University-Level Data

In [ ]:
fig = go.Figure()

univs = df['INSTNM'].unique().tolist() # Get list of unique universities

# Create Violin plot for each University
for univ in univs:
    fig.add_trace(go.Violin(x=df['INSTNM'][df['INSTNM'] == univ],
                            y=df['EARN_MDN_4YR'][df['INSTNM'] == univ],
                            name=univ,
                            box_visible=True,
                            meanline_visible=True,
                            showlegend=False))

# Format the plot
fig.update_yaxes(tickformat="$,.0f")
fig.update_layout(
    title="Earnings 4 Years after Completion Grouped by University",
    yaxis_title="Earnings",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
fig = go.Figure()

univs = df['INSTNM'].unique().tolist() # Get list of unique universities

# Create Violin plot for each University
for univ in univs:
    fig.add_trace(go.Violin(x=df['INSTNM'][df['INSTNM'] == univ],
                            y=df['EARN_MDN_1YR'][df['INSTNM'] == univ],
                            name=univ,
                            box_visible=True,
                            meanline_visible=True,
                            showlegend=False))

# Format the plot
fig.update_yaxes(tickformat="$,.0f")
fig.update_layout(
    title="Earnings 1 Years after Completion Grouped by University",
    yaxis_title="Earnings",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)

fig.show()

## Explore earnings by general Field of Study   
> Field of study defined by first two digits of CIPCODE

In [ ]:
# Group earnings for each field of study (summary of all programs with public record)
field_1yr_stats = df.groupby('CIPDEF').agg({'EARN_COUNT_WNE_1YR': 'sum','EARN_MDN_1YR': ['mean', 'min', 'max']}).reset_index()
field_1yr_stats = field_1yr_stats.sort_values(by=('EARN_MDN_1YR', 'mean'), ascending=False).reset_index(drop=True)
field_1yr_stats

In [ ]:
# Group earnings for each field of study (summary of all programs with public record)
field_4yr_stats = df.groupby('CIPDEF').agg({'EARN_COUNT_WNE_4YR': 'sum','EARN_MDN_4YR': ['mean', 'min', 'max']}).reset_index()
field_4yr_stats = field_4yr_stats.sort_values(by=('EARN_MDN_4YR', 'mean'), ascending=False).reset_index(drop=True)
field_4yr_stats

## Visualize Field of Study Data

In [ ]:
fig = go.Figure()

fields = df['CIPFIELD'].unique().tolist() # Get list of unique fields of study

# Create violin plot for each field of study
for field in fields:
    fig.add_trace(go.Violin(x=df['CIPFIELD'][df['CIPFIELD'] == field],
                            y=df['EARN_MDN_4YR'][df['CIPFIELD'] == field],
                            name=field,
                            box_visible=True,
                            meanline_visible=True,
                            showlegend=False))

# Format the plot
fig.update_yaxes(tickformat="$,.0f")
fig.update_layout(
    title="Earnings 4 Years after Completion Grouped by Field of Study",
    yaxis_title="Earnings",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)

fig.show()

In [ ]:
fig = go.Figure()

fields = df['CIPFIELD'].unique().tolist() # Get list of unique fields of study

# Create violin plot for each field of study
for field in fields:
    fig.add_trace(go.Violin(x=df['CIPFIELD'][df['CIPFIELD'] == field],
                            y=df['EARN_MDN_1YR'][df['CIPFIELD'] == field],
                            name=field,
                            box_visible=True,
                            meanline_visible=True,
                            showlegend=False))

# Format the plot
fig.update_yaxes(tickformat="$,.0f")
fig.update_layout(
    title="Earnings 1 Year after Completion Grouped by Field of Study",
    yaxis_title="Earnings",
    font=dict(
        family="Courier New, monospace",
        size=10,
        color="RebeccaPurple"
    )
)

fig.show()

## Explore earnings by general Field of Study within each University

In [ ]:
# Group earnings for each field of study for each university
univ_field_1yr = df.groupby(['INSTNM', 'CIPDEF']).agg({'EARN_COUNT_WNE_1YR': 'sum','EARN_MDN_1YR': ['mean', 'min', 'max']}).reset_index()


In [ ]:
# Group earnings for each field of study for each university
univ_field_4yr = df.groupby(['INSTNM', 'CIPDEF']).agg({'EARN_COUNT_WNE_4YR': 'sum','EARN_MDN_4YR': ['mean', 'min', 'max']}).reset_index()
